In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas

In [2]:
URL = 'https://en.wikipedia.org/wiki/Spider-Man:_No_Way_Home'
response = requests.get(URL)
soup = bs(response.content)
# Print out the HTML
contents = soup.prettify()

In [3]:
table_data = soup.find(class_="infobox vevent")
info_rows = table_data.find_all("tr")
for row in info_rows:
    print(row.prettify())


<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  Spider-Man: No Way Home
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Spider-Man_No_Way_Home_poster.jpg">
   <img alt="Spider-Man No Way Home poster.jpg" class="thumbborder" data-file-height="330" data-file-width="220" decoding="async" height="330" src="//upload.wikimedia.org/wikipedia/en/0/00/Spider-Man_No_Way_Home_poster.jpg" width="220"/>
  </a>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Directed by
 </th>
 <td class="infobox-data">
  <a href="/wiki/Jon_Watts" title="Jon Watts">
   Jon Watts
  </a>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Written by
 </th>
 <td class="infobox-data">
  <div class="plainlist">
   <ul>
    <li>
   

In [4]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

movie_dict = {}
for index, row in enumerate(info_rows):
    if index == 0:
        movie_dict['title'] = row.find('th').get_text()
    elif index == 1:
        continue
    else:
        content_key = row.find('th').get_text(" ", strip=True) 
        content_value = get_content_value(row.find('td'))
        movie_dict[content_key]=content_value

        

print(movie_dict)


{'title': 'Spider-Man: No Way Home', 'Directed by': 'Jon Watts', 'Written by': ['Chris McKenna', 'Erik Sommers'], 'Based on': ['Stan Lee', 'Steve Ditko'], 'Produced by': ['Kevin Feige', 'Amy Pascal'], 'Starring': ['Tom Holland', 'Zendaya', 'Benedict Cumberbatch', 'Jacob Batalon', 'Jon Favreau', 'Jamie Foxx', 'Willem Dafoe', 'Alfred Molina', 'Benedict Wong', 'Tony Revolori', 'Marisa Tomei', 'Andrew Garfield', 'Tobey Maguire'], 'Cinematography': 'Mauro Fiore', 'Edited by': ['Jeffrey Ford', 'Leigh Folsom Boyd'], 'Music by': 'Michael Giacchino', 'Production companies': ['Columbia Pictures', 'Marvel Studios', 'Pascal Pictures'], 'Distributed by': 'Sony Pictures Releasing', 'Release dates': ['December 13, 2021 ( 2021-12-13 ) ( Fox Village Theatre )', 'December 17, 2021 ( 2021-12-17 ) (United States)'], 'Running time': '148 minutes [1]', 'Country': 'United States', 'Language': 'English', 'Budget': '$200 million [2]', 'Box office': '$1.971 billion [3] [4]'}


In [5]:
URL = 'https://en.wikipedia.org/wiki/List_of_Marvel_Cinematic_Universe_films'
response = requests.get(URL)
soup = bs(response.content)
# Print out the HTML
contents = soup.prettify()
print(contents)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Marvel Cinematic Universe films - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"f2129590-3ecb-4772-9efb-48b3718cef3b","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_Marvel_Cinematic_Universe_films","wgTitle":"List of Marvel Cinematic Universe films","wgCurRevisionId":1123694377,"wgRevisionId":1123694377,"wgArticleId":42163310,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Swedish-language sources (sv)","IMDb ID (Cite Mojo) 

In [6]:
movies = soup.select(".wikitable.sortable i")
movies[0].a['title']

'Iron Man (2008 film)'

In [7]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()
        
def get_info_box(url):

    r = requests.get(url)
    soup = bs(r.content)
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    clean_tags(soup)

    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        else:
            header = row.find('th')
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value
            
    return movie_info    

In [8]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Marvel_Cinematic_Universe_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.wikipedia.org/"

movie_info_list = []
for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(index)
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)

0
10
20
30
40
50
